# LEGAL API Utils for filings

In [ ]:
import os
from sqlalchemy import desc
import datedelta
import base64
import uuid

from legal_api import db
from legal_api.core import Filing as CoreFiling
from legal_api.models import Business, Comment, Filing, RegistrationBootstrap, Office, PartyRole
from legal_api.utils.datetime import date, datetime

In [ ]:
LEGAL_URL = os.getenv('LEGAL_URL')
LEGAL_SVC_CLIENT_ID = os.getenv('LEGAL_SVC_CLIENT_ID')
LEGAL_SVC_CLIENT_SECRET = os.getenv('LEGAL_SVC_CLIENT_SECRET')
ACCOUNT_SVC_AUTH_URL = os.getenv('ACCOUNT_SVC_AUTH_URL')
PAYMENT_SVC_URL = os.getenv('PAYMENT_SVC_URL')
PAYMENT_SVC_CLIENT_SYSTEM_FILING_URL = os.getenv('PAYMENT_SVC_CLIENT_SYSTEM_FILING_URL')
PAYMENT_SVC_RETURN_URL = os.getenv('PAYMENT_SVC_RETURN_URL')

In [ ]:
def get_lear_token(verify_ssl = True):
    payload = "grant_type=client_credentials"
    basic_hash = base64.b64encode(f'{LEGAL_SVC_CLIENT_ID}:{LEGAL_SVC_CLIENT_SECRET}'.encode()).decode()
    basic_hash
    headers = {
      'Content-Type': 'application/x-www-form-urlencoded',
      'Authorization': f'Basic {basic_hash}'
    }

    response = requests.request("POST", ACCOUNT_SVC_AUTH_URL, headers=headers, data=payload, verify=verify_ssl)

    lear_token = response.json()['access_token']
    # lear_token
    assert lear_token
    return lear_token

In [ ]:
def get_filings_by_type(business_id: int, filing_type: str):
    """Return the filings of a particular type."""
    filings = db.session.query(Filing). \
        filter(Filing.business_id == business_id). \
        filter(Filing._filing_type == filing_type). \
        order_by(desc(Filing.filing_date)). \
        all()
    return filings

In [ ]:
def get_filing_by_type(business_id: int, filing_type: str):
    """Return one filing of a particular type."""
    filing = db.session.query(Filing). \
        filter(Filing.business_id == business_id). \
        filter(Filing._filing_type == filing_type). \
        order_by(desc(Filing.filing_date)). \
        one_or_none()
    return filing

In [ ]:
def get_office_by_type(business_id: int, office_type: str):
    """Return one office of a particular type."""
    office = db.session.query(Office). \
        filter(Office.business_id == business_id). \
        filter(Office.office_type == office_type). \
        one_or_none()
    return office

In [ ]:
def get_party_role(business_id: int, role: str):
    """Return one party role"""
    party_role = db.session.query(PartyRole). \
        filter(PartyRole.business_id == business_id). \
        filter(PartyRole.role == role). \
        one_or_none()
    return party_role

In [ ]:
def update_in_good_standing(identifier: str, in_good_standing: bool):
    """Update good standing state for a given business"""
    business = Business.find_by_identifier(identifier)
    business.last_ar_date = None
    business.last_ar_year = None

    if in_good_standing:
        business.founding_date = datetime.utcnow()
    else:
        business.founding_date = datetime.utcnow() - datedelta.datedelta(years=2) 

    business.save()

In [ ]:
def create_empty_draft_ia(lear_token: str, payload: dict, verify_ssl = True):
    url = f'{LEGAL_URL}?draft=true'
    
    headers = {
      'Authorization': f'Bearer {lear_token}'
    }

    r = requests.post(url, headers=headers, json=payload, verify=verify_ssl)
    return r

In [ ]:
def submit_ia(lear_token: str, payload: dict, temp_identifier: str, filing_id: int, verify_ssl = True):
    url = f'{LEGAL_URL}/{temp_identifier}/filings/{filing_id}'
    
    headers = {
      'Authorization': f'Bearer {lear_token}'
    }
    
    r = requests.put(url, headers=headers, json=payload, verify=verify_ssl)
    return r

In [ ]:
def get_presigned_url_file_upload(lear_token: str, file_name: str, verify_ssl = True):
    url = LEGAL_URL.replace('/businesses', '')
    url = f'{url}/documents/{file_name}/signatures'
    
    headers = {
      'Authorization': f'Bearer {lear_token}'
    }

    r = requests.get(url, headers=headers, verify=verify_ssl)
    return r

In [ ]:
def upload_file_to_minio(presigned_url: str, data, verify_ssl = True):
    r = requests.put(presigned_url, data=data, verify=verify_ssl)  
    return r

In [ ]:
def remove_office(business_id: int, office_type: str):
    office = get_office_by_type(business_id, office_type)

    if office:
        addresses = office.addresses.all()

        if addresses and len(addresses) > 0:
            for addr in addresses:
                print(f'delete addr type: {addr.address_type}')
                db.session.delete(addr)

        print(f'delete {office_type} office: {office.id}')
        db.session.delete(office)


In [ ]:
def remove_party_role(business_id: int, role: str):
    party_role = get_party_role(business_id, role)
    if party_role:
        party = party_role.party
        if party:
            # note: addresses hanging off party table are automatically cascade deleted so no need to explicitly delete
            print(f'delete {role} party')
            db.session.delete(party)

        print(f'delete {role} party role')
        db.session.delete(party_role)

In [ ]:
def remove_voluntary_dissolution_filing(business_id: int):
    dissolution_filing = get_filing_by_type(business_id, 'dissolution')
    if dissolution_filing:
        dissolution_filing_json = dissolution_filing.json
        assert dissolution_filing_json['filing']['dissolution']['dissolutionType'] == 'voluntary'
        # print(f'filing: {filings[0].json}')

        dissolution_docs = dissolution_filing.documents.all()
        if(dissolution_docs and len(dissolution_docs) > 0):
            print('found dissolution docs to delete')
            for doc in dissolution_docs:
                print(f'delete doc version type: {doc.type}, doc file key: {doc.file_key}')
                db.session.delete(doc.versions[0])
                print(f'delete doc type: {doc.type}, doc file key: {doc.file_key}')
                db.session.delete(doc)
                db.session.delete(doc.versions[0])

        colin_events = dissolution_filing.colin_event_ids
        if(colin_events and len(colin_events) > 0):
            print('found colin events to delete')
            for cevt in colin_events:
                print(f'delete colin event: {cevt.colin_event_id}, lear filing id: {cevt.filing_id}')
                db.session.delete(cevt)

        print(f'vd payment_token: {dissolution_filing._payment_token}, locked: {dissolution_filing.locked}')
        dissolution_filing._payment_token = None # this needs to be done to unlock filing for deletion
        dissolution_filing.save()
        print(f'delete voluntary dissolution filing id: {dissolution_filing.id}')
        db.session.delete(dissolution_filing)   

In [ ]:
def reset_vd_data(identifier: str):
    """Remove/update voluntary dissolution related data to return business to undissolved state."""
    business = Business.find_by_identifier(identifier)
    assert business

    business.dissolution_date = None
    business.state = 'ACTIVE'
    business.state_filing_id = None
    business.save()
    
    remove_office(business.id, 'custodialOffice')
    remove_party_role(business.id, 'custodian')
    remove_voluntary_dissolution_filing(business.id)

In [ ]:
def create_filing_payment_transaction(temp_identifier: str, payment_token: str, verify_ssl = True):
    url = f'{PAYMENT_SVC_URL}/{payment_token}/transactions'
    payment_svc_client_sys_url = PAYMENT_SVC_CLIENT_SYSTEM_FILING_URL.format(temp_identifier = temp_identifier)

    create_transaction_json = {
      'clientSystemUrl': payment_svc_client_sys_url,
      'payReturnUrl': PAYMENT_SVC_RETURN_URL
    }
    
    headers = {
      'Content-Type': 'application/json'
    }
    payload = json.dumps(create_transaction_json, indent=2)

    r = requests.request("POST", url, headers=headers, data=payload, verify=verify_ssl)
    return r